In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Постановка задачи

Необходимо обучить модель, которая должна предсказывать рейтинг отеля по данным сайта Booking на основе имеющихся в датасете данных. Исходные данные - датасет, в котором содержатся сведения о 515 000 отзывов на отели Европы.

Датасет содержит 17 полей со следующей информацией:

<i>hotel_address</i> — адрес отеля;

<i>additional_number_of_scoring</i> — есть некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.

<i>review_date</i> — дата, когда рецензент разместил соответствующий отзыв;

<i>average_score</i> — средний балл отеля, рассчитанный на основе последнего комментария за последний год;

<i>hotel_name</i> — название отеля;

<i>reviewer_nationality</i> — страна рецензента;

<i>negative_review</i> — отрицательный отзыв, который рецензент дал отелю;

<i>review_total_negative_word_counts</i> — общее количество слов в отрицательном отзыв;

<i>positive_review</i> — положительный отзыв, который рецензент дал отелю;

<i>review_total_positive_word_counts</i> — общее количество слов в положительном отзыве.

<i>reviewer_score</i> — оценка, которую рецензент поставил отелю на основе своего опыта;

<i>total_number_of_reviews_reviewer_has_given</i> — количество отзывов, которые рецензенты дали в прошлом;

<i>total_number_of_reviews</i> — общее количество действительных отзывов об отеле;

<i>tags</i> — теги, которые рецензент дал отелю;

<i>days_since_review</i> — количество дней между датой проверки и датой очистки;

<i>lat</i> — географическая широта отеля;

<i>lng</i> — географическая долгота отеля.


### Импортируем библиотеки

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
import nltk   
import scipy.stats as stats
import re
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_extraction.text import CountVectorizer

import category_encoders as ce
# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42

### Напишем вспомогательные функции для действий, которые в коде ноутбука будут встречаться несколько раз

In [ ]:
# Вспомогательная функция для отрисовки графика связи среднего значения рейтинга и значений в целевой колонке

def plot_dependency(data, column, plot_ylabel, plot_title):
    '''
    Функция рисует коробчатую диаграмму распределения оценки по значениям в переданной колонке
    data: DataFrame
    column: str, name of column to analyze 
    plot_xlabel: str, name of X axis in plot
    plot_title: str, name of the plot
    '''
    data = data[data['sample']==1]
    
    sns.boxplot(y = 'reviewer_score', x = column, data=data).set(title = plot_title)
    plt.ylabel = plot_ylabel
    plt.xlabel = 'Выставленный рейтинг'
    plt.xticks(rotation=45)
    

In [ ]:
# задаём уровень значимости
alpha = 0.05 

# функция для принятия решения об отклонении нулевой гипотезы
def decision_hypothesis(p):
    print('p-value = {:.3f}'.format(p))
    if p <= alpha:
        print('p-значение меньше, чем заданный уровень значимости {:.2f}. Отвергаем нулевую гипотезу в пользу альтернативной.'.format(alpha))
    else:
        print('p-значение больше, чем заданный уровень значимости {:.2f}. У нас нет оснований отвергнуть нулевую гипотезу.'.format(alpha))


In [ ]:
# Вспомогательная функция для статистической проверки зависимости итогового признака от категориального признака методом хи-квадрат
# reviewer_score больше похож на категориальный признак, чем на непрерывный, потому что в нем меньше 50 разных значений на 300 000 записей
def chi_2(data, column):
    df = data[data['sample'] == 1]
    table = pd.crosstab(df[column], df['reviewer_score'])

    _, p, _, _ = stats.chi2_contingency(table)
    decision_hypothesis(p)

In [ ]:
def reduce_categories_number(df, col, n, name):
    '''
    Функция принимает датасет df, колонку col и n - количество наиболее частых категорий.
    Категории не входящие в n наиболее частых заменяются на 'Other'.
    Функция возвращает полученный датасет.
    '''
    popular_values = df[col].value_counts().nlargest(n).index
    df[col] = df[col].apply(lambda x: x if x in popular_values else name)
    return df

In [ ]:
# Вспомогательная функция, очищающая поле tags от лишних символов
def clean_tags(tags):
    tags = tags.replace('[', '')
    tags = tags.replace(']', '')
    tags = tags.replace("'", '')
    return tags.split(',')

In [ ]:
## функция кодирует методом OHE переданную в нее колонку и возвращает новый датасет
def one_hot_manual(df, column, drop_last: True):
    '''
    df: датасет, в котором надо произвести изменения
    column: колонка, которую надо закодировать
    drop_last: удалять ли последнюю колонку из закодированных для уменьшения признаков (последняя колонка все равно описывается нулями 
    во всех предыдущих колонках)
    '''
    for_modify = pd.get_dummies(df[column])
    to_drop = ''
    for col in for_modify.columns:
        for_modify[col] = for_modify[col].apply(lambda x: 1 if x else 0)
        to_drop = col
    if drop_last:
        for_modify = for_modify.drop(to_drop, axis = 1)
    df = pd.concat([df, for_modify], axis = 1, join = 'inner')
    df = df.drop(column, axis = 1)
    return df


### Скачиваем данные

In [ ]:
# DATA_DIR = '/kaggle/input/sf-booking/'
# EXTRA_DATA_DIR = '/kaggle/input/dataset/datasets/'
# EXTRA_DATA_DIR_OTHER = '/kaggle/input/dataset/datasets/Other/'

downloaded_df = pd.read_csv('data/hotels.csv')

from sklearn.model_selection import train_test_split

# Делим данные на train_data (75%) и test_data (25%) из 1 файла
df_train, df_test = train_test_split(downloaded_df, test_size=0.25)

# df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения

df_train = df_train.drop_duplicates()

# df_test = pd.read_csv(DATA_DIR+'/hotels_test.csv') # датасет для предсказания
# sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн


In [ ]:
# для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

# обьединяем датасеты в один, для более удобной работы с одной датасетом, а не двумя.
data = pd.concat([df_train, df_test], ignore_index = True)

## <font color = blue>1. Изучим пропуски в данных</font>

In [ ]:
data.info()

Видно, что пропуски есть только в полях широты и долготы отеля. Изучем эти поля поподробнее

In [ ]:
#Найдем отели, в которых пропущены и lat и lng
data['lat_is_null'] = data['lat'].isnull()
data['lng_is_null'] = data['lng'].isnull()
mask1 = data['lat_is_null'] == True
mask2 = data['lng_is_null'] == True
hotels_lat_lng_is_null = data[mask1 & mask2][['hotel_address', 'hotel_name', 'lat', 'lng']]
hotels_lat_lng_is_null.groupby(['hotel_address', 'hotel_name']).mean()

In [ ]:
# Создадим датафрейм с координатами (полученными с помощью Google Maps по наименованиям и адресам):

hotels_coord = {
    'hotel' : [
        '20 Rue De La Ga t 14th arr 75014 Paris France', 
        '23 Rue Damr mont 18th arr 75018 Paris France', 
        '4 rue de la P pini re 8th arr 75008 Paris France', 
        'Bail n 4 6 Eixample 08010 Barcelona Spain', 
        'Gr nentorgasse 30 09 Alsergrund 1090 Vienna Austria',
        'Hasenauerstra e 12 19 D bling 1190 Vienna Austria',
        'Josefst dter Stra e 10 12 08 Josefstadt 1080 Vienna Austria',
        'Josefst dter Stra e 22 08 Josefstadt 1080 Vienna Austria',
        'Landstra er G rtel 5 03 Landstra e 1030 Vienna Austria',
        'Paragonstra e 1 11 Simmering 1110 Vienna Austria',
        'Pau Clar s 122 Eixample 08009 Barcelona Spain',
        'Savoyenstra e 2 16 Ottakring 1160 Vienna Austria',
        'Sep lveda 180 Eixample 08011 Barcelona Spain',
        'Sieveringer Stra e 4 19 D bling 1190 Vienna Austria',
        'Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Austria',
        'W hringer Stra e 12 09 Alsergrund 1090 Vienna Austria',
        'W hringer Stra e 33 35 09 Alsergrund 1090 Vienna Austria'
    ],
    'lat' : [48.838861, 48.888909, 48.875317, 41.391548, 48.220852, 48.233588, 48.209402, 
             48.209549, 48.188818, 48.222761, 41.392685, 48.219569, 41.384199, 48.246018, 
             48.213587, 48.216800, 48.220410],    
    'lng' : [2.318447, 2.330578, 2.320817, 2.175319, 11.488682, 11.467736, 16.350871,
            16.348888, 16.381204, 16.390912, 2.164823, 16.283017, 2.148535, 16.338658,
            16.377333, 16.357331, 16.353244]
}
hotels_coord_df = pd.DataFrame(hotels_coord)
hotels_coord_df

In [ ]:
def lat_func(hotel_address):
    for i in range(17):
        if hotel_address == hotels_coord_df['hotel'][i]:
            hotel_lat = hotels_coord_df['lat'][i]
            return hotel_lat

data['hotel_lat'] = data['hotel_address'].apply(lat_func)

def lng_func(hotel_address):
    for i in range(17):
        if hotel_address == hotels_coord_df['hotel'][i]:
            hotel_lng = hotels_coord_df['lng'][i]
            return hotel_lng

data['hotel_lng'] = data['hotel_address'].apply(lng_func)

# Заполним пропущенные данные в столбцах lat и lat данными с координатами центров городов city_lat и city_lng:

data['lat'] = data['lat'].fillna(data['hotel_lat'])
data['lng'] = data['lng'].fillna(data['hotel_lng'])

data.info()

Теперь пропусков в ячейках не осталось.

In [ ]:
# Избавимся от лишних ячеек
data = data.drop(['hotel_lat', 'hotel_lng', 'lat_is_null', 'lng_is_null'], axis = 1)
data.info()

## <font color = blue>2. Изучим признак "адрес отеля"</font>

In [ ]:
sns.set(font_scale = 1)
# создадим новый признак страны
data['country'] = data['hotel_address'].apply(lambda x: x.split()[-1]\
                                            if x.split()[-1] != 'Kingdom'\
                                                    else ' '.join(x.split()[-2:]))
# создадим новый признак города 
data['city'] = data['hotel_address'].apply(lambda x: x.split()[-2]\
                                         if x.split()[-1] != 'Kingdom'\
                                         else x.split()[-5])
#Проверим частотность городов и стран
data.groupby(['city']).count().plot( 
    kind='pie', y = 'hotel_address', autopct='%1.0f%%') 
data.groupby(['country']).count().plot( 
    kind='pie', y = 'hotel_address', autopct='%1.0f%%') 

## Видно, что адреса отелей находятся в 6 столицах 6 стран. Это значит, что из адреса отеля можно оставить только один признак. Пусть это будет country

In [ ]:
#Удаляем ненужные признаки об адресе отеля
data = data.drop(['hotel_address', 'city'], axis = 1)
data.head(3)

## Посмотрим зависимость средней оценки от страны, в которой находится отель

In [ ]:
plot_dependency(data[data['sample'] == 1], 'country', 'Страна', 'Оценка по стране')

## Рисуется странная картина: есть разница между Австрией и Испанией и четырьмя другими странами. Имеет смысл преобразовать признак так, чтобы он указывал на "Австрию-Испанию" и "Другое"

In [ ]:
def spain_or_not(country):
    if country in ('Austria', 'Spain'):
        return 1
    return 0

data['Spain_Austria'] = data['country'].apply(spain_or_not)
data.head(3)

Признак country оставляем, так как он может нам потребоваться дальше.

## <font color = blue>3. Изучим название отеля</font>

In [ ]:
# формируем список сетей отелей, действующих в Eвропе
hotels_chain_list = ['Hilton', 'Marriott', 'Holiday Inn', 'Hyatt', 'Sheraton',
                     'Premier Inn', 'Jurys Inns', 'Banyan', 'Seasons', 'Rixos', 
                     'Best Western', 'Crowne Plaza', 'Radisson', 'Ritz-Carlton',
                     'Best Western', 'Britannia Hotels ', 'Ibis', 'Kempinski', 
                     'Meridien', 'Ramada', 'Sofitel', 'Mercure', 'NH', 'Novotel',
                     'Macdonald', 'Ramada', 'Grange Hotels', 'Wetherspoon', 
                     'B&b', 'Campanile', 'Logis', 'Kyriad', 'Disney', 'Citotel',
                     'Collectionneurs', 'Brit', 'Mgallery', 'Fasthotel', 'Relais',
                     'Contact Hôtel', 'Starhotels', 'Gruppo Una', 'Blu Hotels', 
                     'Apogia', 'Iti', 'Leonardi', 'Charme E Relax', 'H10', 
                     'Meliá', 'Barceló', 'Eurostars', 'Senator', 'Catalonia', 
                     'Iberostar', 'Paradores', 'Ilunion', 'Tryp', 'Marriott',
                     'Exe', 'Silken', 'Vincci', 'Petit', 'Sercotel', 'Globales', 
                     'Austria Trend', 'Jufa', 'Privatecityhotels', 'Van Der Valk',
                     'Fletcher', 'Bastion', 'Golden Tulip']

# сформируем признак и обозначим отели принадлежащие к сети, как "1", в противном случае "0"
def get_hotel_cain(hotel_name):
    for name_cain in hotels_chain_list:
        if name_cain in hotel_name:
            return 1
    return 0
# выведем результат 
data['in_hotel_chain'] = data['hotel_name'].apply(get_hotel_cain)
plot_dependency(data, 'in_hotel_chain', 'Отель из сети отелей', 'Зависимость оценки от нахождения отеля в сети')

По графику видно, что оценка отеля вроде бы зависит от того, входит ли отель в какую-либо сеть. 

Проверим это статистически, методом хи-квадрат.

<b>Нулевая гипотеза:</b> признак нахождения в сети отелей и выставленная оценка не зависят друг от друга.

<b>Альтернативная гипотеза:</b> признаки зависимы.

In [ ]:
chi_2(data, 'in_hotel_chain')

Оставляем признак вхождения отеля в сеть отелей. 

## <font color = blue>4. Изучим признак даты</font>

In [ ]:
# Превратим колонку даты в тип даты
data['review_date'] = pd.to_datetime(data['review_date'])
# возьмем день недели и месяц
data['review_day_of_week'] = data['review_date'].dt.day_of_week
data['review_month'] = data['review_date'].dt.month
data.head(3)

In [ ]:
# Удалим колонку даты
data = data.drop(['review_date'], axis = 1)
data.head(3)

## <font color = blue>4.1 Изучим признак дня недели</font>

In [ ]:
plot_dependency(data[data['sample']==1], 'review_day_of_week', 'День недели', 'Оценка по дням недели')


Коробки с усами почти идентичны для каждого дня недели

Проверим статистически, есть ли зависимость выставленной оценки от дня недели, при помощи критерия хи-квадрат.

**Нулевая гипотеза:** признаки независимы

**Альтернативная гипотеза:** признаки зависимы

In [ ]:
chi_2(data, 'review_day_of_week')


Оставляем признак дня недели. 

## <font color = blue>4.2 Изучим признак месяц выставления оценки</font>

In [ ]:
plot_dependency(data[data['sample']==1], 'review_month', 'Месяц', 'Средняя оценка по месяцам')


Видно, что выставленная оценка слегка варьируется в зависимости от месяца. Проверим, может быть, она сильнее зависит от времени года?

In [ ]:
def return_season(month):
    if month in (12, 1, 2):
        return 1
    elif month in (3, 4, 5):
        return 2
    elif month in (6, 7, 8):
        return 3
    return 4

data['season'] = data['review_month'].apply(return_season)
data.head(3)

In [ ]:
plot_dependency(data[data['sample'] == 1], 'season', 'Сезон', 'Средняя оценка по сезонам')

**Похоже, что зависимость от сезона прослеживается довольно слабо: имеет смысл, зима это или любой другой сезон. По предыдущему графику видно, что декабрь практически не отличается от других месяцев.**

**Поэтому сделаем так же, как и со страной: введем признак "январь-февраль" это или другой месяц. Остальные признаки удалим.****

In [ ]:
def is_Jan(month):
    if month in (1, 2):
        return 1
    return 0

data['is_Jan_or_Feb'] = data['review_month'].apply(is_Jan)
data.head(3)

Удалим ненужные колонки из датасета

In [ ]:
data = data.drop(['review_month', 'season'], axis=1)
data.head(3)

## <font color = blue>5. Проверим признак национальности ревьюера</font>

In [ ]:
# Проверим зависимость от того, одинакова ли страна отеля и государство гостя
data['local_reviewer'] = data.apply(lambda x: 1 if str(x['reviewer_nationality']).strip() == x['country'] else 0, axis=1)
plot_dependency(data, 'local_reviewer', 'Национальность совпадает', 'Зависимость оценки от совпадения национальности')

Визуально оценка отеля мало зависит от того, совпадает национальность гостя со страной отеля или нет. 

Проверим статистически при помощи критерия хи-квадрат. 

**Нулевая гипотеза:** взаимосвязи между признаками нет.

**Альтернативная гипотеза:** взаимосвязь между признаками есть.

In [ ]:
chi_2(data, 'local_reviewer')

Зависимость есть. Оставляем признак

## Проверим, а есть ли вообще зависимость оценки от национальности гостя

In [ ]:
# Посмотрим, сколько всего национальностей у гостей
data['reviewer_nationality'].nunique()

Слишком много. Уменьшим до 10

In [ ]:
data = reduce_categories_number(data, 'reviewer_nationality', 10, 'Other_nat')
data.head()

Посмотрим на распределение оценок по выделенным категориям

In [ ]:
plot_dependency(data, 'reviewer_nationality', 'Национальность', 'Зависимость оценки от национальности гостя')

Судя по графику, национальность ревьюера оказывает влияние на выставленную оценку. Проверим эту гипотезу статистически.

Поскольку у нас два категориальных признака - национальность ревьюера и его оценка - и поскольку в каждом из признаков больше 5 категорий, можно воспользоваться критерием хи-квадрат.

**Нулевая гипотеза:** признаки независимы

**Альтернативная гипотеза:** признаки зависимы



In [ ]:
chi_2(data, 'reviewer_nationality')

И статистический анализ, и визуальный говорит, что национальность ревьюера имеет значение. Оставляем признак.

## <font color = blue> 6. Изучим признак тэгов.</font>

In [ ]:
# Подсчитаем частотность значений в колонке tags
# для этого сначала составим словарь из всех возможных значений поля tags, а затем подсчитаем, сколько этих значений максимально в одной ячейке

tags_collection = []
tags_count = {}
tags_figure = 0

def count_tags(tags, tags_figure):
    # очищаем поле tags от "мусора"
    tags_list = clean_tags(tags)
    f = len(tags_list)
    # Добавляем элемент в список, если его там еще нет
    for el in tags_list:
        if el.strip() in tags_collection:
            tags_count[el.strip()] += 1
        else:
            tags_collection.append(el.strip())
            tags_count[el.strip()] = 1
    # Определяем максимальный размер списка тэгов в одной ячейке
    if f > tags_figure:
        return f
    else:
        return tags_figure       
  
# Считаем максимальное количество тэгов в одной ячейке и заодно заполняем список уникальных тэгов
tags_figure = data['tags'].apply(lambda x: count_tags(x, tags_figure))

print(f'Max tags count for 1 cell = {tags_figure.max()}')
print(f'Count of unique tags = {len(tags_collection)}')
tags_count = {k: v for k, v in sorted(tags_count.items(), key = lambda item: item[1], reverse = True)}
print(tags_count)



## <font color = blue>6.1 Оценим влияние типа поездки</font>

In [ ]:
## Выделим тип поездки из поля тэгов 

def extract_trip(tags):
    tags_list = clean_tags(tags)
    for el in tags_list:
        if 'trip' in el:
            return el.strip()
    return 'Not a trip'

data['trip'] = data['tags'].apply(extract_trip)
data.head(3)

In [ ]:
plot_dependency(data, 'trip', "Тип поездки", "Зависимость оценки от типа поездки")

Видно, что есть зависимость выставленной оценки от типа поездки. Проверим статистически.

In [ ]:
chi_2(data, 'trip')

Тип поездки оказывает влияние на выставленную оценку. Оставляем признак.


## <font color = blue>6.2 Оценим влияние того, что оценка выставлена с телефона</font>

In [ ]:
data['from_mobile'] = data['tags'].apply(lambda x: 1 if ' mobile ' in x else 0)
plot_dependency(data, 'from_mobile', 'Оценка с телефона', 'Зависимость оценки от того, с телефона ли она проведена')

Визуально различий между категориями признака нет. Проверим статистически.

**Нулевая гипотеза** Оценка отеля не зависит от того, с мобильного телефона она выставлена или нет

**Альтернативная гипотеза** Признаки зависимы

In [ ]:
chi_2(data, 'from_mobile')

Оставляем признак, так как он оказывает влияние.

## <font color = blue>6.3 Оценим влияние на оценку количества ночей, которые гость провел в отеле</font>

In [ ]:
# Выделим из признака тэгов количество ночей, которые гость оставался в отеле.

def extract_nights(tags):
    tags_list = clean_tags(tags)
    for el in tags_list:
        if 'Stayed' in el:
            result = re.findall(r'\d+', el)
            return str(result[0]) + 'nights'
    return 0

data['nights'] = data['tags'].apply(extract_nights)
data['nights'].value_counts()

In [ ]:
# Уменьшим количество ночей
data = reduce_categories_number(data, 'nights', 10, 'Other_night')
data.head(3)

In [ ]:
plot_dependency(data, 'nights', 'Количество ночей', 'Зависимость оценки от количества ночей в отеле')

Визуально различий между категориями признака почти нет. Проверим статистически.

**Нулевая гипотеза** Оценка отеля не зависит от того, сколько ночей клиент провел в отеле

**Альтернативная гипотеза** Признаки зависимы

In [ ]:
chi_2(data, 'nights')

**Оставляем признак**

## <font color = blue>6.4. Изучим влияние типа номера на оценку</font> 

In [ ]:
def extract_room(tags):
    tags_list = clean_tags(tags)
    for el in tags_list:
        if 'Room' in el:
            return el.strip()
    return 'Unknown'

data['rooms'] = data['tags'].apply(extract_room)

data = reduce_categories_number(data, 'rooms', 10, 'Other_room')
data['rooms'].value_counts()

In [ ]:
plot_dependency(data, 'rooms', 'Тип комнаты', 'Зависимость отзыва от типа комнаты')

Видно, что оценка довольно сильно зависит от того, в какой комнате жил гость. Проверим статистически.

**Нулевая гипотеза** Оценка отеля не зависит от того, в какой комнате жил гость

**Альтернативная гипотеза** Признаки зависимы

In [ ]:
chi_2(data, 'rooms')

Выставленная оценка зависит от типа комнаты, в которой жил ревьюер

## <font color = blue>6.5 Изучим влияние количества выставленных тэгов на оценку </font>

In [ ]:
def tag_count(tags):
    tags_list = clean_tags(tags)
    return str(len(tags_list)) + ' tags'

data['tags_count'] = data['tags'].apply(tag_count)
data['tags_count'].head(10)

In [ ]:
plot_dependency(data, 'tags_count', 'Число тэгов', 'Зависимость оценки от числа тэгов')

Видно, что оценка, вероятно, зависит от того, сколько тэгов поставил гость. Проверим статистически.

**Нулевая гипотеза** Оценка отеля не зависит от того, сколько тэгов поставлено

**Альтернативная гипотеза** Признаки зависимы

In [ ]:
chi_2(data, 'tags_count')

**Оставляем признак**

## <font color = blue>7. Проанализируем текст оценки</font>

In [ ]:
# Начнем с позитивного текста оценки
data['positive_review'].value_counts().head(25)

In [ ]:
#Проверим, содержит ли позитивный текст оценки что-то вроде No positive или nothing
def extract_positive(review):
    if 'no positive' in review.lower() or 'nothing' in review.lower():
        return 0
    return 1

#введем новый признак, есть ли позитивный текст у оценки
data['is_positive'] = data['positive_review'].apply(extract_positive)
data['is_positive'].value_counts()

In [ ]:
# теперь проверим негативный отзыв точно так же
data['negative_review'].value_counts().head(25)

In [ ]:
# Проверим, содержит ли текст негативной оценки слова, отрицающие негативную оценку
def extract_negative(review):
    if 'no negative' in review.lower() or 'nothing' in review.lower() or 'n a' in review.lower() or 'none' in review.lower():
        return 0
    if 'nil' in review.lower() or 'no complaints' in review.lower() or 'all good' in review.lower():
        return 0
    return 1

# Заведем новый признак, есть ли негатив в негативном отзыве
data['is_negative'] = data['negative_review'].apply(extract_negative)
data['is_negative'].value_counts()

In [ ]:
# Загрузим корпус данных для анализа тональности:

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
nltk.downloader.download('vader_lexicon')

sent_analyzer = SentimentIntensityAnalyzer()

# Используем метод polarity_scores для анализа тональности негативных и позитивных отзывов. 
# Результаты запишем в столбцы neg_sent и pos_sent. Также, для того, чтобы эту информацию можно 
# было использовать в обучении модели, создадим отдельные признаки со значениями neg, neu, pos 
# и compound, которые определяют степень отрицательности, нейтральности, положительности и 
# общую тональность текста соответственно:

data['neg_sent'] = data['negative_review'].apply(lambda x: sent_analyzer.polarity_scores(x))

data['neg_sent_neg'] = data['neg_sent'].apply(lambda x: x['neg'])
data['neg_sent_neu'] = data['neg_sent'].apply(lambda x: x['neu'])
data['neg_sent_pos'] = data['neg_sent'].apply(lambda x: x['pos'])
data['neg_sent_cmpnd'] = data['neg_sent'].apply(lambda x: x['compound'])

data['pos_sent'] = data['positive_review'].apply(lambda x: sent_analyzer.polarity_scores(x))

data['pos_sent_neg'] = data['pos_sent'].apply(lambda x: x['neg'])
data['pos_sent_neu'] = data['pos_sent'].apply(lambda x: x['neu'])
data['pos_sent_pos'] = data['pos_sent'].apply(lambda x: x['pos'])
data['pos_sent_cmpnd'] = data['pos_sent'].apply(lambda x: x['compound'])
data.info()

In [ ]:
# Удалим лишние признаки
data = data.drop(['neg_sent', 'pos_sent'], axis = 1)

## <font color = blue>8. Числовые признаки</font>

In [ ]:
#Извлекаем числовой признак из колонки days_since_review
data['days_since_review'] = data['days_since_review'].apply(lambda x: int((str(x).replace(' day', '')).replace('s', '')))
data.head(3)

Видим, что колонки total_number_of_reviews и additional_number_of_scoring имеют значение в сотни и даже тысячи, в то время как колонки, полученные только что при оценке степени положительности и отрицательности отзыва, варьируются от 0 до 1. 

Нормализуем колонки

In [ ]:
# Сначала преобразуем колонку additional_number_of_scoring
data['number_scoring_normal'] = (data['additional_number_of_scoring']-data['additional_number_of_scoring'].mean())/data['additional_number_of_scoring'].std()
data.head(3)

In [ ]:
# Потом колонку total_number_of_reviews
data['total_number_normal'] = (data['total_number_of_reviews']-data['total_number_of_reviews'].mean())/data['total_number_of_reviews'].std()
data.head(3)

In [ ]:
#Потом колонку total_number_of_reviews_reviewer_has_given
data['total_rev_number_normal'] = (data['total_number_of_reviews_reviewer_has_given']-data['total_number_of_reviews_reviewer_has_given'].mean())/data['total_number_of_reviews_reviewer_has_given'].std()
data.head(3)

In [ ]:
#Удалим ненужные колонки 
data = data.drop(['additional_number_of_scoring', 'total_number_of_reviews', 'hotel_name', 'negative_review', 
                  'positive_review', 'total_number_of_reviews_reviewer_has_given'], axis = 1)
data.head(3)

In [ ]:
#Осталась ненормализованной колонка days_since_review. Нормализуем ее
data['days_normal'] = (data['days_since_review']-data['days_since_review'].mean())/data['days_since_review'].std()
data.head(3)

In [ ]:
#Еще раз удалим лишние ячейки
data = data.drop(['tags', 'days_since_review'], axis = 1)
data.head(3)

## <font color = blue>9. Перекодировка категориальных признаков методонм ОНЕ</font>

In [ ]:
/# Перекодируем категориальные признаки при помощи вспомогательной функции

df = one_hot_manual(data, 'trip', False)
df = one_hot_manual(df, 'tags_count', False)
df = one_hot_manual(df, 'reviewer_nationality', True)
df = one_hot_manual(df, 'review_day_of_week', True)
df = one_hot_manual(df, 'nights', True)
df = one_hot_manual(df, 'rooms', True)
df.info()

In [ ]:
# Переименуем колонки с днями недели, сейчас их тип int и модель такое название не поймет
df.rename(columns = {0:'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 
                     4:'Friday', 5: 'Saturday', 6: 'Sunday'}, inplace = True)

df.info()

In [ ]:
# Удалим лишние ячейки
df = df.drop(['country'], axis = 1)
df.head(3)

## <font color = blue>10. Проверка на мультиколлинеарность</font>

In [ ]:
# проверим корреляцию итоговых признаков
def chart_correlation_heatmap(df, 
                              columns, 
                              title, 
                              method='pearson', 
                              correlation_threshold=.6):
    """Построение корреляционной диаграммы"""
    fig_, ax_ = plt.subplots(figsize=(70, 70))
    correlation_matrix = df[columns].corr(method=method).abs()
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix[correlation_matrix.abs() > correlation_threshold], 
                annot=True,
                annot_kws={"fontsize":16}, 
                linewidths=0.1, 
                ax=ax_, 
                mask=mask, 
                cmap='GnBu',
                fmt='.2f')
    ax_.set_title(title, fontsize=24)
    plt.show()

In [ ]:
sns.set(font_scale=2)
chart_correlation_heatmap(df, 
                          df.columns, 
                          'Кореляция числовых признаков',
                          correlation_threshold = .7)

Видно, что корреляцией 0.7 или выше обладают следующие признаки:

Spain_Austria и lat

United Kingdom и local_reviewer

4 tags и from_mobile

5 tags и from_mobile

pos_sent_neg и is_positive

pos_sent_cmpnd и neg_sent_neg

neg_sent_neu и neg_sent_neg

pos_sent_pos и pos_sent_neu

total_number_normal и number_scoring_normal

Leisure trip и Business trip

5 tags и 4 tags

Проверим, какие из них оказывают большее влияние на признак оценки


In [ ]:
# Проверим вклад колонок в итоговые признаки, чтобы определить, какие признаки больше влияют на результат
num_cols = ['total_number_normal', 'number_scoring_normal', 'pos_sent_neg', 'pos_sent_cmpnd', 
           'neg_sent_neg', 'pos_sent_pos', 'pos_sent_neu', 'neg_sent_neu']
cat_cols = ['Spain_Austria', 'lat', ' United Kingdom ', 'local_reviewer', '4 tags', '5 tags', 'from_mobile',
           'is_positive', 'Leisure trip', 'Business trip']

# Теперь выделим тестовую часть
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Используем непараметрический тест хи-квадрат, реализованный в библиотеке sklearn.feature_selection.chi2. Метод возвращает 
# массив значений хи-квадрат и p-value для каждого признака. Используем только значения хи-квадрат и выведем их на графике:

y = y.astype('int')

from sklearn.feature_selection import chi2 # хи-квадрат

imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Удалим признаки lat, 5 tags, local_reviewer, from_mobile, Leisure trip

In [ ]:
# Используем функцию f_classif из библиотеки sklearn. В основе метода оценки значимости переменных лежит анализ ANOVA. 
# Основу процедуры составляет обобщение результатов двух выборочных t-тестов для независимых выборок (2-sample t).

# Метод возвращает двумерный массив f-статистик и p-value для каждого признака. В качестве меры значимости используем значение 
# f-статистики. Чем значение статистики выше, тем меньше вероятность того, что средние значения не отличаются, и тем важнее данный 
# признак для нашей модели.

from sklearn.feature_selection import f_classif # anova

imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Удалим признаки pos_sent_neu, number_scoring_normal, is_positive, neg_sent_neg

In [ ]:
df = df.drop(['pos_sent_neu', 'number_scoring_normal', 'is_positive', 'neg_sent_neg',
             'lat', '5 tags', 'local_reviewer', 'from_mobile', 'Leisure trip'], axis = 1)

In [ ]:
sns.set(font_scale=2)
chart_correlation_heatmap(df, 
                          df.columns, 
                          'Кореляция числовых признаков',
                          correlation_threshold = .7)

**Избавились от корреляции в признаках**

## <font color = blue>11. Построение модели</font>

In [ ]:
# Теперь выделим тестовую часть
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

model = RandomForestRegressor(
    n_estimators=100, 
    verbose=1, 
    n_jobs=-1, 
    random_state=RANDOM_SEED)

In [ ]:
# обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)
predict_submission = model.predict(test_data)
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)